In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
#import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/output_pln_usd.csv")

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 7.5 MB/s 


In [ ]:
import keras_tuner as kt

In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
#len(df_numpy[:2500])
scaler1 = scaler.fit(df_numpy[:3001])
df_scalled = scaler1.transform(df_numpy)
#df_scaled_all = scaler.
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
df_train = df_scalled[2100:3001]
df_val = df_scalled[1700:2100] #300
df_game = df_scalled[3001:]

In [ ]:
lookback = 30
step = 1
delay = 30
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')

In [ ]:
float_game_data = np.array(df_game).astype('float32')

In [ ]:
len(float_train_data)

901

In [ ]:
len(float_game_data)

541

In [ ]:
len(float_val_data)

400

In [ ]:
float_train_data = np.array(df_train).astype('float32')

In [ ]:
float_val_data = np.array(df_val).astype('float32')

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
train_gen = generator(float_train_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=900 - delay,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_val_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=399 - delay,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=3001,
max_index=3541 - delay,
step=step,
batch_size=batch_size)

In [ ]:
val_steps = (399 - lookback)
test_steps = (3541 - 3001 - lookback)

In [ ]:
del model

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(120,
  activation='sigmoid',
 # dropout=0.2,
  #recurrent_dropout=0.1,
  input_shape=(None, float_train_data.shape[-1])))
model.add(layers.Dense(160,
  activation='sigmoid',))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=200,
  epochs=200,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/200
200/200 [==============================] - 28s 129ms/step - loss: 0.1707 - val_loss: 0.2202
Epoch 2/200
200/200 [==============================] - 25s 127ms/step - loss: 0.0851 - val_loss: 0.1112
Epoch 3/200
200/200 [==============================] - 25s 126ms/step - loss: 0.0679 - val_loss: 0.0601
Epoch 4/200
200/200 [==============================] - 28s 141ms/step - loss: 0.0571 - val_loss: 0.0842
Epoch 5/200
200/200 [==============================] - 25s 126ms/step - loss: 0.0512 - val_loss: 0.0763
Epoch 6/200
200/200 [==============================] - 25s 127ms/step - loss: 0.0489 - val_loss: 0.0744
Epoch 7/200
200/200 [==============================] - 26s 128ms/step - loss: 0.0480 - val_loss: 0.0702
Epoch 8/200
200/200 [==============================] - 25s 127ms/step - loss: 0.0457 - val_loss: 0.0763
Epoch 9/200
200/200 [==============================] - 25s 127ms/step - loss: 0.0446 - val_loss: 0.0693
Epoch 10/200
200/200 [==============================] - 25s 127m

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

510/510 [==============================] - 17s 33ms/step - loss: 0.0566


In [ ]:
model.save("drive/MyDrive/Engineer's Project/longer_prediction.h5")

In [ ]:
model = tf.keras.models.load_model("drive/MyDrive/Engineer's Project/standard_model.h5")

In [ ]:
del model

In [ ]:
len(float_game_data)

542

In [ ]:
correct_picks = 0
value = 0
for i in range(541-30): #len(float_train_data - lookback - delay
  if len(float_game_data) <= 30:
    break
  last = float_train_data[-lookback:]
  last = last.reshape(1,30,7)
  prediction = model.predict(last)
  last_val = last[0][lookback-1][3]
  actual_val = float_game_data[30][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_picks+=1
    value += 100 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_picks+=1
    value += 100 * (last_val - actual_val)
  else:
    value -= 100 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

In [ ]:
correct_picks

300

In [ ]:
value

327.56914496421814

In [ ]:
#updated model
print(correct_picks)
print(value)

298
4589.046776294708


version 2 updates the model

In [ ]:
def return_model():
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(120,
    activation='sigmoid',
    dropout=0.2,
    recurrent_dropout=0.1,
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.Dense(80,
    activation='sigmoid',))
  model.add(layers.Dense(1,
    activation='sigmoid'))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
  return model

In [ ]:
len(float_train_data)

931

In [ ]:
correct_picks = 0
value = 0
for i in range(0,542-30):
  if len(float_game_data) <= 30:
    break
  last = float_train_data[-lookback:]
  last = last.reshape(1,30,7)
  prediction = model.predict(last)
  last_val = last[0][lookback-1][3]
  actual_val = float_game_data[29][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_picks+=1
    value += 1000 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_picks+=1
    value += 1000 * (last_val - actual_val)
  else:
    value -= 1000 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

  if i % 30 == 0:
    del model
    del train_gen
    model = return_model()
    train_gen = generator(float_train_data,
      lookback=lookback,
      delay=delay,
      min_index=0,
      max_index=len(float_train_data) - delay - 1,
      #shuffle=True,
      step=step,
      batch_size=batch_size)
    model.fit(train_gen,
      steps_per_epoch=200,
      epochs=20,
      validation_data=val_gen,
      validation_steps=val_steps)

Epoch 1/20
200/200 [==============================] - 25s 114ms/step - loss: 0.1046 - val_loss: 0.2210
Epoch 2/20
200/200 [==============================] - 23s 114ms/step - loss: 0.0801 - val_loss: 0.0763
Epoch 3/20
200/200 [==============================] - 23s 114ms/step - loss: 0.0647 - val_loss: 0.0771
Epoch 4/20
200/200 [==============================] - 23s 114ms/step - loss: 0.0606 - val_loss: 0.0909
Epoch 5/20
200/200 [==============================] - 23s 114ms/step - loss: 0.0576 - val_loss: 0.0949
Epoch 6/20
200/200 [==============================] - 23s 114ms/step - loss: 0.0558 - val_loss: 0.0779
Epoch 7/20
200/200 [==============================] - 23s 113ms/step - loss: 0.0554 - val_loss: 0.0815
Epoch 8/20
200/200 [==============================] - 23s 113ms/step - loss: 0.0539 - val_loss: 0.0836
Epoch 9/20
200/200 [==============================] - 23s 114ms/step - loss: 0.0533 - val_loss: 0.0788
Epoch 10/20
200/200 [==============================] - 23s 113ms/step - l

In [ ]:
correct_picks

272

In [ ]:
value

875.434935092926